In [1]:
import cPickle as pickle
import mxnet as mx
import numpy as np
from data_utilities import load_mnist
import mx_layers as layers
from mx_utility import mxnet_array_mapping
from symbolic_gradient import _filter_arguments

In [2]:
parameters, states = pickle.load(open('parameters/mnist-baseline-network', 'rb'))
_, _, _, _, data, labels = load_mnist(path='shrinked_mnist', shape=(1, 28, 28))

In [3]:
def _normalized_convolution(**args):
  network = layers.convolution(**args)
  network = layers.batch_normalization(network)
  network = layers.ReLU(network)
  return network

In [4]:
network = layers.variable('data')
network = _normalized_convolution(X=network, n_filters=16, kernel_shape=(5, 5), stride=(1, 1), pad=(2, 2))
network = layers.pooling(X=network, mode='maximum', kernel_shape=(2, 2), stride=(2, 2), pad=(0, 0))
for index in range(3):
  identity = network
  residual = _normalized_convolution(X=network, n_filters=16, kernel_shape=(3, 3), stride=(1, 1), pad=(1, 1))
  network = identity + residual
network = layers.pooling(X=network, mode='average', global_pool=True, kernel_shape=(28, 28), stride=(1, 1), pad=(0, 0))
network = layers.flatten(network)
network = layers.fully_connected(X=network, n_hidden_units=10)

In [5]:
parameters, states = _filter_arguments(network, parameters, states)
context = mx.cpu()
parameters = mxnet_array_mapping(parameters, context)
states = mxnet_array_mapping(states, context)
parameters['data'] = mx.nd.array(data, context)

In [6]:
executor = network.bind(context, parameters, aux_states=states, grad_req='null')

In [7]:
scores = executor.forward()[0]

In [8]:
np.count_nonzero(np.argmax(scores.asnumpy(), axis=1) - labels) / float(len(data))

0.6934